# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [66]:
# Load environment variables
from dotenv import load_dotenv
import os
from glob import glob

load_dotenv()

# Load the environment variable
price_data_path = os.getenv("PRICE_DATA")

# Check if it loaded correctly
print("PRICE_DATA:", price_data_path)





PRICE_DATA: ../../05_src/data/prices/


In [67]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [68]:
parquet_files = glob(os.path.join(price_data_path, "*", "*", "part.*.parquet"))

print(f"Found {len(parquet_files)} parquet files")
print("Sample paths:", parquet_files[:5])



Found 3158 parquet files
Sample paths: ['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [69]:
# Add lag features
ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)

# Add return feature
ddf["returns"] = (ddf["Close"] / ddf["Close_lag_1"]) - 1

# Add high-low range feature
ddf["hi_lo_range"] = ddf["High"] - ddf["Low"]

# Assign the result
dd_feat = ddf


C:\Users\scott\AppData\Local\Temp\ipykernel_14872\2654294188.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
C:\Users\scott\AppData\Local\Temp\ipykernel_14872\2654294188.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [88]:
import dask.dataframe as dd
import pandas as pd

# Step 1: Load the sample into a Dask DataFrame
ddf = dd.read_parquet(parquet_sample)

# Step 2: Create new features
ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)
ddf["returns"] = (ddf["Close"] / ddf["Close_lag_1"]) - 1
ddf["hi_lo_range"] = ddf["High"] - ddf["Low"]

# Step 3: Convert Dask DataFrame to pandas
df_feat = ddf.compute()

# Step 4: Add 10-day moving average of returns
df_feat["returns_ma_10"] = df_feat.groupby("ticker")["returns"].transform(lambda x: x.rolling(10).mean())

# Step 5: Preview the result
df_feat.head()


C:\Users\scott\AppData\Local\Temp\ipykernel_14872\182602804.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
C:\Users\scott\AppData\Local\Temp\ipykernel_14872\182602804.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10
137371,2001-10-12,15.26,16.549999,15.260000,16.549999,12.441836,5193300.0,ACN.csv,ACN,2001,15.160000,11.396875,0.091689,1.289999,NaN
137374,2001-10-17,17.65,17.740000,16.850000,16.980000,12.765102,1877100.0,ACN.csv,ACN,2001,17.450001,13.118434,-0.026934,0.889999,NaN
137377,2001-10-22,17.00,17.570000,16.969999,17.500000,13.156021,469600.0,ACN.csv,ACN,2001,16.850000,12.667368,0.038576,0.600000,NaN
137379,2001-10-24,17.68,17.760000,17.570000,17.740000,13.336446,472600.0,ACN.csv,ACN,2001,17.620001,13.246236,0.006810,0.190001,NaN
137380,2001-10-25,17.74,17.750000,17.129999,17.740000,13.336446,265100.0,ACN.csv,ACN,2001,17.740000,13.336446,0.000000,0.620001,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Yes, it was necessary to convert to pandas because Dask does not yet support the .rolling() method in combination with groupby operations. While Dask is optimized for large datasets, pandas offers more flexible time series functionality like rolling windows. If Dask had full rolling support within groups, it would have been more efficient to stay in Dask to avoid loading everything into memory.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.